In [2]:
import mediapipe as mp
import cv2 as cv
import numpy as np
import math
import time


In [3]:
draw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
def angle_calculator(x,y):
    j=(-1)**0.5
    a=x[0]+x[1]*j
    b=y[0]+y[1]*j
    k=(x[0]*y[0]+x[1]*y[1])/(abs(a)*abs(b))
    theta=np.arccos([k])*180/math.pi

    return theta


In [40]:
vid=cv.VideoCapture('KneeBendVideo.mp4')          #video capturing for provided video


i=0
s='0'                                             # these are the variables required in the codes
flag=True
prev_i=0                                          
r_angle=0
cur=0
count=0
count_time=[0,0,0]
min_angle=[180]

f=cv.VideoWriter_fourcc(*'mp4v')
cap=cv.VideoWriter('knee_bending.mp4',f,20,(854,640))


with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.7) as pose:
    
    
    prev=time.time()                                 ##for calculating time
    while vid.isOpened():                                         
        ret,frame=vid.read()  
        if ret:
            frame=cv.resize(frame,(854,640))

            kernel=np.array([[0,-1,0],
                             [-1,5,-1],
                             [0,-1,0]])
            frame=cv.filter2D(frame,-1,kernel)                     ##sharpening the frames
            frame=cv.bilateralFilter(frame,5,41,100)               ##applying blurr effect
            img=cv.cvtColor(frame,cv.COLOR_BGR2RGB)                ##converting video from bgr to rgb scale for mediapipe
#             cv.imshow('img',img)
            
            result=pose.process(img)
        
        
            if result.pose_landmarks:


                left_hip_x=result.pose_landmarks.landmark[mp_pose.PoseLandmark(23).value].x       ##x and y coordinates for calculating
                left_knee_x=result.pose_landmarks.landmark[mp_pose.PoseLandmark(25).value].x      ## vector so as to calculate angle
                left_ankle_x=result.pose_landmarks.landmark[mp_pose.PoseLandmark(27).value].x     ##between hip-knee vector and
                                                                                                  ##ankle-knee vector
                left_hip_y=result.pose_landmarks.landmark[mp_pose.PoseLandmark(23).value].y
                left_knee_y=result.pose_landmarks.landmark[mp_pose.PoseLandmark(25).value].y
                left_ankle_y=result.pose_landmarks.landmark[mp_pose.PoseLandmark(27).value].y


                hip_vec_x=left_hip_x-left_knee_x
                hip_vec_y=-1*(left_hip_y-left_knee_y)

                ankle_vec_x=left_ankle_x-left_knee_x
                ankle_vec_y=-1*(left_ankle_y-left_knee_y)

                angle=angle_calculator([hip_vec_x,hip_vec_y],[ankle_vec_x,ankle_vec_y])          
                if float(s)>2:
                    min_angle=min(angle+10,min_angle)

            cv.rectangle(frame,(35,65),(170,95),[255,0,0],-1)                                 ##creating rectangle and putting text
            cv.rectangle(frame,(500,50),(750,80),[0,255,255],-1)                              ## for display
            cv.putText(frame,"Stats: ",(510,75),cv.FONT_HERSHEY_COMPLEX_SMALL,1.7,[255,125,127],2)
            cv.putText(frame,"Counts: "+str(count),(510,110),cv.FONT_HERSHEY_SIMPLEX,0.65,[255,255,0],2)
            cv.putText(frame,"Angle: "+str(int(min_angle[0])),(510,140),cv.FONT_HERSHEY_SIMPLEX,0.65,[255,255,0],2)


            cv.rectangle(frame,(500,150),(770,190),[0,200,0],-1)
            cv.putText(frame,"Last Record: ",(510,185),cv.FONT_HERSHEY_COMPLEX_SMALL,1.7,[100,0,100],2)
            cv.putText(frame,"Min Angle: "+str(r_angle),(510,220),cv.FONT_HERSHEY_SIMPLEX,0.65,[255,255,0],2)
            cv.putText(frame,"Time: "+str(count_time[0]),(510,250),cv.FONT_HERSHEY_SIMPLEX,0.65,[255,255,0],2)



            if angle<140:                                                                ## aplying given conditions

                cv.circle(frame,(50,80),12,[0,255,0],-1)
                cur=time.time()
                prev_i=i

            else:
                if abs(prev_i-i)>1:
                    count_time.insert(0,float("{:.2f}".format(prev_i+(cur-prev)-1)))         ## keeping records for last three
                    count_time.pop()                                                         ## intervals of time
                    if 0<count_time[0]<8 :
                        flag=False
                    else:
                        flag=True

                    if flag:
                        count+=1

                    prev_i=i
                    r_angle=int(min_angle[0])
                    min_angle=[180]


                i=0    
                cv.putText(frame,str(i),(75,90),cv.FONT_HERSHEY_SIMPLEX,1,[255,255,0],2)
                cv.circle(frame,(50,80),12,[0,0,255],-1)


            # for displaying warning 

            if 0<count_time[0]<8 and angle>140:
                cv.rectangle(frame,(35,120),(370,160),[0,200,255],-1)
                cv.putText(frame,str("Keep your knee bent"),(35,150),cv.FONT_HERSHEY_COMPLEX_SMALL,1.25,[255,0,255],2)



            if(cur-prev)<1 and angle<140:
                s="{:.2f}".format(i+(cur-prev)-1)                                             ##for displaying time
                if float(s)<0:
                    cv.putText(frame,str(0),(75,90),cv.FONT_HERSHEY_SIMPLEX,1,[255,255,0],2)
                else:
                    cv.putText(frame,str(s),(75,90),cv.FONT_HERSHEY_SIMPLEX,1,[255,255,0],2)


            if (cur-prev)>1:                                                                  
                    prev=time.time()
                    i+=1




            draw.draw_landmarks(
                frame,
                result.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,

            )

            cv.imshow('frame',frame)
            cap.write(frame)
            if cv.waitKey(1) & 0XFF==ord('q'):
                break
        else:
            break
            
vid.release()
cap.release()
cv.destroyAllWindows()

In [84]:
vid.release()
cv.destroyAllWindows()